In [4]:
%pip install mlflow

  Using cached mlflow-2.21.3-py3-none-any.whl.metadata (30 kB)
Using cached mlflow-2.21.3-py3-none-any.whl (28.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from abc import ABC, abstractmethod
import mlflow
import mlflow.sklearn
import pandas as pd

In [7]:
class Evaluation(ABC):
    @abstractmethod
    def calculate_scores(self, y_true, y_pred):
        pass

class MSE(Evaluation):
    def calculate_scores(self, y_true, y_pred):
        return mean_squared_error(y_true, y_pred)

class R2(Evaluation):
    def calculate_scores(self, y_true, y_pred):
        return r2_score(y_true, y_pred)


In [8]:
data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
X_train, X_test, y_train, y_test = train_test_split(df, data.target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = MSE()
r2 = R2()
mse_score = mse.calculate_scores(y_test, y_pred)
r2_score_value = r2.calculate_scores(y_test, y_pred)



In [9]:
input_example = pd.DataFrame([X_test.iloc[0]]) 
mlflow.set_experiment("diabetes_evaluation")
mlflow.end_run() 
with mlflow.start_run():
    mlflow.log_metric("mse", mse_score)
    mlflow.log_metric("r2", r2_score_value)
    mlflow.sklearn.log_model(model, "model", input_example=input_example)
with mlflow.start_run():
    mlflow.log_metric("mse", mse_score)
    mlflow.log_metric("r2", r2_score_value)
    mlflow.sklearn.log_model(model, "model", input_example=input_example)

print("MSE Score:", mse_score)
print("R2 Score:", r2_score_value)


MSE Score: 2900.1936284934814
R2 Score: 0.4526027629719195
